In [33]:
from fvcore.nn import FlopCountAnalysis, parameter_count_table
import torch
from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present

from hifigan.generator import HifiganGenerator

In [6]:
dependencies = ["torch", "torchaudio"]

URLS = {
    "hifigan": "https://github.com/bshall/hifigan/releases/download/v0.1/hifigan-67926ec6.pt",
    "hifigan-hubert-discrete": "https://github.com/bshall/hifigan/releases/download/v0.1/hifigan-hubert-discrete-bbad3043.pt",
    "hifigan-hubert-soft": "https://github.com/bshall/hifigan/releases/download/v0.1/hifigan-hubert-soft-65f03469.pt",
}


def _hifigan(
        name: str,
        pretrained: bool = True,
        progress: bool = True,
        map_location=None,
) -> HifiganGenerator:
    hifigan = HifiganGenerator()
    if pretrained:
        checkpoint = torch.hub.load_state_dict_from_url(
            URLS[name], map_location=map_location, progress=progress
        )
        consume_prefix_in_state_dict_if_present(checkpoint, "module.")
        hifigan.load_state_dict(checkpoint)
        hifigan.eval()
        hifigan.remove_weight_norm()
    return hifigan


def hifigan(
        pretrained: bool = True,
        progress: bool = True,
        map_location=None,
) -> HifiganGenerator:
    """HiFiGAN Vocoder from from `"A Comparison of Discrete and Soft Speech Units for Improved Voice Conversion"`.
    Args:
        pretrained (bool): load pretrained weights into the model
        progress (bool): show progress bar when downloading model
        map_location: a function or a dict specifying how to remap storage locations (see torch.load)
    """
    return _hifigan("hifigan", pretrained, progress, map_location)


def hifigan_hubert_soft(
        pretrained: bool = True,
        progress: bool = True,
        map_location=None,
) -> HifiganGenerator:
    """HiFiGAN Vocoder from from `"A Comparison of Discrete and Soft Speech Units for Improved Voice Conversion"`.
    Finetuned on spectrograms generated from the soft acoustic model.
    Args:
        pretrained (bool): load pretrained weights into the model
        progress (bool): show progress bar when downloading model
        map_location: a function or a dict specifying how to remap storage locations (see torch.load)
    """
    return _hifigan(
        "hifigan-hubert-soft", pretrained, progress, map_location=map_location
    )


def hifigan_hubert_discrete(
        pretrained: bool = True,
        progress: bool = True,
        map_location=None,
) -> HifiganGenerator:
    """HiFiGAN Vocoder from from `"A Comparison of Discrete and Soft Speech Units for Improved Voice Conversion"`.
    Finetuned on spectrograms generated from the discrete acoustic model.
    Args:
        pretrained (bool): load pretrained weights into the model
        progress (bool): show progress bar when downloading model
        map_location: a function or a dict specifying how to remap storage locations (see torch.load)
    """
    return _hifigan(
        "hifigan-hubert-discrete", pretrained, progress, map_location=map_location
    )


In [7]:
model = hifigan_hubert_discrete()

In [8]:


def get_module_size(module, prefix=""):
    """Récursivement calcule la taille de chaque sous-module d'un modèle PyTorch en mégaoctets."""
    submodule_sizes = {}
    total_size = 0

    # Parcours des sous-modules
    for name, submodule in module.named_children():
        # Calcul de la taille des paramètres du sous-module en octets
        submodule_size, sizes = get_module_size(submodule, prefix=f"{prefix}{name}.")
        submodule_sizes.update(sizes)
        total_size += submodule_size

    # Taille du module actuel
    module_size = sum(p.numel() * p.element_size() for p in module.parameters())
    buffer_size = sum(b.numel() * b.element_size() for b in module.buffers())
    current_size = module_size + buffer_size  # Taille totale en octets pour ce module
    total_size += current_size

    # Enregistrer la taille du module actuel
    submodule_sizes[f"{prefix}{module.__class__.__name__}"] = current_size / (1024 * 1024)  # Conversion en mégaoctets

    return total_size, submodule_sizes


def print_module_sizes(module):
    """Affiche les tailles des sous-modules en mégaoctets."""
    _, sizes = get_module_size(module)
    print("\nTaille des sous-modules (en MB) :\n")
    for name, size in sizes.items():
        print(f"{name}: {size:.2f} MB")


# Affiche les tailles des sous-modules
print_module_sizes(model)



Taille des sous-modules (en MB) :

conv_pre.Conv1d: 1.75 MB
ups.0.ConvTranspose1d: 10.00 MB
ups.1.ConvTranspose1d: 1.00 MB
ups.2.ConvTranspose1d: 0.13 MB
ups.3.ConvTranspose1d: 0.03 MB
ups.ModuleList: 11.16 MB
resblocks.0.convs1.0.Conv1d: 0.75 MB
resblocks.0.convs1.1.Conv1d: 0.75 MB
resblocks.0.convs1.2.Conv1d: 0.75 MB
resblocks.0.convs1.ModuleList: 2.25 MB
resblocks.0.convs2.0.Conv1d: 0.75 MB
resblocks.0.convs2.1.Conv1d: 0.75 MB
resblocks.0.convs2.2.Conv1d: 0.75 MB
resblocks.0.convs2.ModuleList: 2.25 MB
resblocks.0.ResBlock: 4.51 MB
resblocks.1.convs1.0.Conv1d: 1.75 MB
resblocks.1.convs1.1.Conv1d: 1.75 MB
resblocks.1.convs1.2.Conv1d: 1.75 MB
resblocks.1.convs1.ModuleList: 5.25 MB
resblocks.1.convs2.0.Conv1d: 1.75 MB
resblocks.1.convs2.1.Conv1d: 1.75 MB
resblocks.1.convs2.2.Conv1d: 1.75 MB
resblocks.1.convs2.ModuleList: 5.25 MB
resblocks.1.ResBlock: 10.51 MB
resblocks.2.convs1.0.Conv1d: 2.75 MB
resblocks.2.convs1.1.Conv1d: 2.75 MB
resblocks.2.convs1.2.Conv1d: 2.75 MB
resblocks.2.convs

In [38]:
light_model = HifiganGenerator(
    in_channels=128,
    resblock_dilation_sizes=((1, 3, 5), (1, 3, 5), (1, 3, 5)),
    resblock_kernel_sizes=(3, 7, 11),
    upsample_kernel_sizes=(16, 16, 4, 4),
    upsample_initial_channel=256,
    upsample_factors=(8, 8, 2, 2),
    sample_rate=16000
)
flops = FlopCountAnalysis(light_model, torch.randn(1, 128, 800)).total()
print(f"Flops: {flops / 1e9:.2f} G")
print(parameter_count_table(light_model, max_depth=2))

/home/hadware/Code/ogmios-workbench/hifigan/venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Unsupported operator aten::_weight_norm encountered 78 time(s)
Unsupported operator aten::leaky_relu encountered 77 time(s)
Unsupported operator aten::add encountered 36 time(s)
Unsupported operator aten::add_ encountered 8 time(s)
Unsupported operator aten::div encountered 4 time(s)
Unsupported operator aten::tanh encountered 1 time(s)


Flops: 61.55 G
| name                 | #elements or shape   |
|:---------------------|:---------------------|
| model                | 3.6M                 |
|  conv_pre            |  0.2M                |
|   conv_pre.bias      |   (256,)             |
|   conv_pre.weight_g  |   (256, 1, 1)        |
|   conv_pre.weight_v  |   (256, 128, 7)      |
|  ups                 |  0.7M                |
|   ups.0              |   0.5M               |
|   ups.1              |   0.1M               |
|   ups.2              |   8.3K               |
|   ups.3              |   2.1K               |
|  resblocks           |  2.8M                |
|   resblocks.0        |   0.3M               |
|   resblocks.1        |   0.7M               |
|   resblocks.2        |   1.1M               |
|   resblocks.3        |   74.5K              |
|   resblocks.4        |   0.2M               |
|   resblocks.5        |   0.3M               |
|   resblocks.6        |   18.8K              |
|   resblocks.7        | 

[device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type='cpu'),
 device(type=